In [1]:
# default_exp spiral

# python

In [2]:
import util

# node

In [3]:
%%node export

import * as zmq from "zeromq"

var requestRun = async (prev: Promise<string | null>, file: any): Promise<string | null> => {
    const port: number = 13805
    await prev // Waiting on the previous request is so they get ordered properly. Otherwise, messages might fill up and fire in arbitrary order.
    const sock = new zmq.Request()
    const uriServer = `tcp://localhost:${port}`
    sock.connect(uriServer)
    await sock.send(JSON.stringify(file))
    const [x] = await sock.receive()
    sock.disconnect(uriServer)
    return x ? x.toString() : null
}
var prev_request: Promise<string | null> = new Promise(resolve => resolve(null))
var request = async (file: any) => { prev_request = requestRun(prev_request, file); return prev_request }
var requestJSON = (file: any) => request(file).then(x => x ? JSON.parse(x) : undefined)

var spiBuildFileReq = async (uri: string, backend: string): Promise<void> => requestJSON({ BuildFile: { uri, backend } })


01490d225a6646acac7358905f39113c


<IPython.core.display.Javascript object>

'spiral.ts'

In [4]:
%%node export

import * as fs from "fs"
import * as util from "./util"

export var spiToFsx = async (spiPath: string) => {
    const fsxPath = spiPath.replace('.spi', '.fsx')
    await spiBuildFileReq(spiPath, 'Fsharp')
    await util.waitFileChange(fsxPath)
    const lines = fs.readFileSync(fsxPath).toString().split('\n')
    const [imports, code] = lines.reduce(
        ([imports, code], line) =>
            /^(open|\#r) .*?$/.test(line)
                ? [[...imports, line], code]
                : [imports, [...code, line]],
            [[], []]
    )
    fs.writeFileSync(fsxPath, [...imports, '', ...code].join('\n'))
}

'spiral.ts'

In [5]:
%%node run

await spiToFsx('/workspaces/spiral-notebook/main.spi')
util.logStep('build_spiral end')


['0 s, 54.474 ms - build_spiral end']